In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [2]:
import pandas as pd

# Read the JSON file into a dataframe
df = pd.read_json(r"/home/jovyan/thesis/vec.json")

In [3]:
X = df['RNN vectors'].to_list() 
y = df['Encoded Categories'].to_list() 

max_len = max(len(x) for x in X)# Fix Shape
X = np.array([np.concatenate([x, [0] * (max_len - len(x))]) for x in X])

features = X
target = y

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)

rf.fit(X_train, y_train)

y_pred_rnn = rf.predict(X_test)
f1score_rnn = f1_score(y_test, y_pred_rnn, average='weighted')
print(f"F1-score: {f1score_rnn}")

F1-score: 0.053190714589171274


In [4]:
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, roc_auc_score, f1_score

In [8]:
# Calculate confusion matrix
cm_rnn = confusion_matrix(y_test, y_pred_rnn)
print("Confusion matrix:")
print(cm_rnn)

Confusion matrix:
[[334   0   0 ...   0   0   0]
 [  3   0   0 ...   0   0   0]
 [  2   0   0 ...   0   0   0]
 ...
 [  2   0   0 ...   0   0   0]
 [  1   0   0 ...   0   0   0]
 [  1   0   0 ...   0   0   0]]


In [10]:
# Calculate accuracy
acc_rnn = accuracy_score(y_test, y_pred_rnn)
print(f"Accuracy: {acc_rnn}")

Accuracy: 0.16133584195672623


In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [7]:
print("Classification report:")
print(classification_report(y_test, y_pred_rnn))

Classification report:
              precision    recall  f1-score   support

           0       0.17      0.96      0.29       349
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         3
           5       0.00      0.00      0.00         4
           9       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         2
          11       0.00      0.00      0.00         1
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         1
          18       0.00      0.00      0.00         1
          25       0.00      0.00      0.00         1
          26       0.00      0.00      0.00         5
          29       0.00      0.00      0.00         1
          31       0.00      0.00      0.00         1
          32       0.00      0.00      0.00         1
    

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
!pip install gensim


[notice] A new release of pip available: 22.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [10]:
import pandas as pd
import dask.dataframe as dd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

def doc2vec_pdftext(df):
    # Convert pdftext column to list of TaggedDocument objects
    tagged_data = [TaggedDocument(words=doc.split(), tags=[str(i)]) for i, doc in enumerate(df['pdftext'].tolist())]

    # Convert list to Dask DataFrame
    ddf = dd.from_pandas(pd.DataFrame(tagged_data), npartitions=4)

    # Train Doc2Vec model
    model = Doc2Vec(vector_size=50, min_count=2, epochs=40, workers=4)
    model.build_vocab(ddf.values.compute())

    for epoch in range(40):
        model.train(ddf.values.rechunk(1000).compute(), total_examples=model.corpus_count, epochs=1)
        model.alpha -= 0.002
        model.min_alpha = model.alpha

    # Get embeddings and convert to list of lists
    embeddings = model.docvecs.vectors_docs.tolist()

    return embeddings

# Apply function to pdftext column
df['doc2vec_embeddings'] = doc2vec_pdftext(df)

AttributeError: 'numpy.ndarray' object has no attribute 'words'

In [29]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [31]:
#doc2vec
import numpy as np
import gensim

def doc2vec(documents):

    #documents = ['this is the first document',             'this document is the second document',             'and this is the third one',             'is this the first document']

    tagged_documents = [gensim.models.doc2vec.TaggedDocument(doc.split(), [i]) for i, doc in enumerate(documents)]

    model = gensim.models.Doc2Vec(tagged_documents, vector_size=100, window=5, min_count=1, workers=4)

    doc_vectors = model.docvecs

    # You can get the vector representation of a document using the following command

    return np.array(doc_vectors).flatten()
df['doc2vec vectors']=df['sentences'].apply(doc2vec)

/tmp/ipykernel_15727/912405552.py:13: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  doc_vectors = model.docvecs


In [ ]:
print('hi')

In [ ]:
df['doc2vec_vectors'][0]

In [ ]:
scores = cross_val_score(rf, X, y, cv=5)

print(f"Cross-validation scores: {scores}")
print(f"Average cross-validation score: {np.mean(scores)}")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

X = df['LSA vectors'].to_list()  # Features (PDF document vectors)
y = df['Encoded Categories'].to_list()  # Target variable (categories)

# Fix Shape as vectors are of different shapes
max_len = max(len(x) for x in X)
X = np.array([np.concatenate([x, [0] * (max_len - len(x))]) for x in X])

features = X
target = y

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)

rf.fit(X_train, y_train)

y_pred_lsa = rf.predict(X_test)
f1score_lsa = f1_score(y_test, y_pred_lsa, average='weighted')
print(f"F1-score: {f1score_lsa}")


In [ ]:
# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:")
print(cm)

# Calculate accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc}")

# Calculate ROC curve and ROC AUC score
y_proba = rf.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
roc_auc = roc_auc_score(y_test, y_proba)
print("ROC curve:")
print(fpr, tpr)
print(f"ROC AUC score: {roc_auc}")

# Calculate F1 score
f1score = f1_score(y_test, y_pred, average='weighted')
print(f"F1 score: {f1score}")